In [1]:
from pathlib import Path
from random import shuffle

# from evaluate import load as load_metric
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator
from miditok.utils import split_files_for_training
from miditok.data_augmentation import augment_dataset
from torch import Tensor, argmax
from torch.utils.data import DataLoader
from torch.cuda import is_available as cuda_available, is_bf16_supported
from torch.backends.mps import is_available as mps_available
from transformers import (
    AutoModelForCausalLM,
    GPT2Config,
    Trainer,
    TrainingArguments,
)
from transformers.trainer_utils import set_seed

In [2]:
# Seed
set_seed(777)

# Our tokenizer's configuration
BEAT_RES = {(0, 1): 12, (1, 2): 4, (2, 4): 2, (4, 8): 1}
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": BEAT_RES,
    "num_velocities": 24,
    "special_tokens": ["PAD", "BOS", "EOS"],
    "use_chords": True,
    "use_rests": True,
    "use_tempos": False,
    "use_time_signatures": False,
    "use_programs": False,  # no multitrack here
    "num_tempos": 32,
    "tempo_range": (50, 200),  # (min_tempo, max_tempo)
}
config = TokenizerConfig(**TOKENIZER_PARAMS)

# Creates the tokenizer
tokenizer = REMI(config)

file_name = "filtered-midi-files"
# Trains the tokenizer with Byte Pair Encoding (BPE) to build the vocabulary, here 30k tokens
midi_paths = list(
    Path("../data/filtered-midi-files").resolve().glob("**/*.mid")
) + list(Path("../data/filtered-midi-files").resolve().glob("**/*.midi"))
print(midi_paths)

[WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/(Sittin On) The Dock Of The Bay.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/American Beauty.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Boogie Woogie Santa.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Breathing.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Cascades.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Chicago Breakdown.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Cleanin Up Christmas.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Cleopha.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/minibach/data/filtered-midi-files/Blues/Combination March.mid'), WindowsPath('C:/Users/mikol/mini/MiniBach/mini

In [3]:
tokenizer.train(
    vocab_size=20000,
    files_paths=midi_paths,
)
tokenizer.save_params("tokenizer_filtered_reduced_half.json")

# tokenizer_path = Path("tokenizer_filtered_reduced.json")
# tokenizer = REMI(params=tokenizer_path)

C:\Users\mikol\AppData\Local\Temp\ipykernel_13480\4049761988.py:5: UserWarning: miditok: The `save_params` method had been renamed `save`. It is now depreciated and will be removed in future updates.
  tokenizer.save_params("tokenizer_filtered_reduced_half.json")


In [4]:
print(len(tokenizer))

20000


In [5]:
# Split MIDI paths in train/valid/test sets
total_num_files = len(midi_paths)
num_files_valid = round(total_num_files * 0.15)
num_files_test = round(total_num_files * 0.15)
shuffle(midi_paths)
midi_paths_valid = midi_paths[:num_files_valid]
midi_paths_test = midi_paths[num_files_valid : num_files_valid + num_files_test]
midi_paths_train = midi_paths[num_files_valid + num_files_test :]

# # Chunk MIDIs and perform data augmentation on each subset independently
for files_paths, subset_name in (
    (midi_paths_train, "train"),
    (midi_paths_valid, "valid"),
    (midi_paths_test, "test"),
):

    # Split the MIDIs into chunks of sizes approximately about 1024 tokens
    subset_chunks_dir = Path(f"filtered_midi/Maestro_{subset_name}")
    split_files_for_training(
        files_paths=files_paths,
        tokenizer=tokenizer,
        save_dir=subset_chunks_dir,
        max_seq_len=1024,
        num_overlap_bars=2,
    )

# # Perform data augmentation
# augment_dataset(
#     subset_chunks_dir,
#     pitch_offsets=[-12, 12],
#     velocity_offsets=[-4, 4],
#     duration_offsets=[-0.5, 0.5],
# )

Splitting music files (filtered_midi\Maestro_train): 100%|██████████| 6145/6145 [00:23<00:00, 265.56it/s]
Splitting music files (filtered_midi\Maestro_valid): 100%|██████████| 1317/1317 [00:10<00:00, 121.40it/s]
Splitting music files (filtered_midi\Maestro_test): 100%|██████████| 1317/1317 [00:04<00:00, 321.99it/s]


In [7]:
dir_name = "filtered_midi"
# Create Dataset and Collator for training
midi_paths_train = list(Path(f"{dir_name}/Maestro_train").glob("**/*.mid")) + list(
    Path(f"{dir_name}/Maestro_train").glob("**/*.midi")
)
midi_paths_valid = list(Path(f"{dir_name}/Maestro_valid").glob("**/*.mid")) + list(
    Path(f"{dir_name}/Maestro_valid").glob("**/*.midi")
)
midi_paths_test = list(Path(f"{dir_name}/Maestro_test").glob("**/*.mid")) + list(
    Path(f"{dir_name}/Maestro_test").glob("**/*.midi")
)
kwargs_dataset = {
    "max_seq_len": 1024,
    "tokenizer": tokenizer,
    "bos_token_id": tokenizer["BOS_None"],
    "eos_token_id": tokenizer["EOS_None"],
}
dataset_train = DatasetMIDI(midi_paths_train, **kwargs_dataset)
dataset_valid = DatasetMIDI(midi_paths_valid, **kwargs_dataset)
dataset_test = DatasetMIDI(midi_paths_test, **kwargs_dataset)

In [8]:
print(dataset_train)

46056 files.


In [9]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
model_config = GPT2Config(
    vocab_size=len(tokenizer),
    n_embd=768,
    n_inner=1024,
    n_layer=12,
    n_head=12,
    n_positions=2048,
    n_ctx=1024,
    pad_token_id=tokenizer["PAD_None"],
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

model = AutoModelForCausalLM.from_config(model_config)

In [12]:
from evaluate import load as load_metric

metrics = {metric: load_metric(metric) for metric in ["accuracy"]}


def compute_metrics(eval_pred):
    """
    Compute metrics for pretraining.

    Must use preprocess_logits function that converts logits to predictions (argmax or sampling).

    :param eval_pred: EvalPrediction containing predictions and labels
    :return: metrics
    """
    predictions, labels = eval_pred
    not_pad_mask = labels != -100
    labels, predictions = labels[not_pad_mask], predictions[not_pad_mask]
    return metrics["accuracy"].compute(
        predictions=predictions.flatten(), references=labels.flatten()
    )


def preprocess_logits(logits: Tensor, _: Tensor) -> Tensor:
    """
    Preprocess the logits before accumulating them during evaluation.

    This allows to significantly reduce the memory usage and make the training tractable.
    """
    pred_ids = argmax(logits, dim=-1)  # long dtype
    return pred_ids

In [ ]:
# Create config for the Trainer
USE_CUDA = cuda_available()
if not cuda_available():
    FP16 = FP16_EVAL = BF16 = BF16_EVAL = False
elif is_bf16_supported():
    BF16 = BF16_EVAL = True
    FP16 = FP16_EVAL = False
else:
    BF16 = BF16_EVAL = False
    FP16 = FP16_EVAL = True
USE_MPS = not USE_CUDA and mps_available()
training_config = TrainingArguments(
    "filtered_midi",
    True,
    True,
    False,
    False,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=3.0,
    max_steps=10000,
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.3,
    log_level="debug",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    save_steps=2000,
    save_total_limit=5,
    no_cuda=not USE_CUDA,
    seed=444,
    fp16=FP16,
    bf16=BF16,
    label_smoothing_factor=0.0,
    optim="adamw_torch",
    report_to=["tensorboard"],
    gradient_checkpointing=True,
)

collator = DataCollator(tokenizer["PAD_None"], copy_inputs_as_labels=True)
trainer = Trainer(
    model=model,
    args=training_config,
    data_collator=collator,
    train_dataset=dataset_train,
    compute_metrics=compute_metrics,
    callbacks=None,
    preprocess_logits_for_metrics=preprocess_logits,
)

# Training
train_result = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
Currently training with a batch size of: 4
***** Running training *****
  Num examples = 46,056
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 10,000
  Number of trainable parameters = 64,215,552


  0%|          | 0/10000 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 10.0485, 'grad_norm': 25.025156021118164, 'learning_rate': 6.666666666666667e-07, 'epoch': 0.0}
{'loss': 10.0058, 'grad_norm': 19.512693405151367, 'learning_rate': 1.3333333333333334e-06, 'epoch': 0.01}
{'loss': 9.9395, 'grad_norm': 22.386886596679688, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 9.8753, 'grad_norm': 19.597890853881836, 'learning_rate': 2.666666666666667e-06, 'epoch': 0.01}
{'loss': 9.7999, 'grad_norm': 12.81788444519043, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.02}
{'loss': 9.7506, 'grad_norm': 11.897374153137207, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 9.6807, 'grad_norm': 10.399171829223633, 'learning_rate': 4.666666666666667e-06, 'epoch': 0.02}
{'loss': 9.6649, 'grad_norm': 8.415386199951172, 'learning_rate': 5.333333333333334e-06, 'epoch': 0.03}
{'loss': 9.5973, 'grad_norm': 8.56799602508545, 'learning_rate': 6e-06, 'epoch': 0.03}
{'loss': 9.527, 'grad_norm': 10.134136199951172, 'learning_rate': 6.666